In [ ]:
import vaex
import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime
from astropy.table import Table
from matplotlib import pyplot as plt

In [ ]:
files = glob("<combined files path>")
files.sort()

In [ ]:
full = vaex.open_many(files)

In [ ]:
# masking

cek_gaia = full.rv_gaia == full.rv_gaia
cek_gaia_no = full.rv_gaia != full.rv_gaia

cek_rave = full.rv_rave == full.rv_rave
cek_rave_no = full.rv_rave != full.rv_rave

cek_lamost = full.rv_lamost == full.rv_lamost
cek_lamost_no = full.rv_lamost != full.rv_lamost

cek_apogee = full.rv_apogee == full.rv_apogee
cek_apogee_no = full.rv_apogee != full.rv_apogee

cek_galah = full.rv_galah == full.rv_galah
cek_galah_no = full.rv_galah != full.rv_galah

In [ ]:
# 5rv, all 5 rv from each catalog must presents
mask = cek_gaia*cek_rave*cek_lamost*cek_apogee*cek_galah
# check how many rows got filtered
mask.sum()

In [ ]:
rv5 = full[mask]
rv5.export("5rv.hdf5", progress=True)

In [ ]:
# 4rv
mask1 = cek_gaia*cek_rave*cek_lamost*cek_apogee
mask2 = cek_gaia*cek_rave*cek_lamost*cek_galah
mask3 = cek_gaia*cek_rave*cek_apogee*cek_galah
mask4 = cek_gaia*cek_lamost*cek_apogee*cek_galah
mask5 = cek_rave*cek_lamost*cek_apogee*cek_galah
mask = mask1 | mask2 | mask3 | mask4 | mask5
mask.sum()

In [ ]:
rv4 = full[mask]
rv4.export("4rv.hdf5", progress=True)

In [ ]:
# 3rv
mask1 = cek_gaia*cek_galah*cek_rave
mask2 = cek_gaia*cek_galah*cek_lamost
mask3 = cek_gaia*cek_galah*cek_apogee
mask4 = cek_gaia*cek_rave*cek_lamost
mask5 = cek_gaia*cek_rave*cek_apogee
mask6 = cek_gaia*cek_lamost*cek_apogee
mask7 = cek_galah*cek_rave*cek_lamost
mask8 = cek_galah*cek_rave*cek_apogee
mask9 = cek_galah*cek_lamost*cek_apogee
mask10 = cek_rave*cek_lamost*cek_apogee

mask = mask1 | mask2 | mask3 | mask4 | mask5 | mask6 | mask7 | mask8 | mask9 | mask10
mask.sum()

In [ ]:
rv3 = full[mask]
rv3.export("3rv.hdf5", progress=True)

In [ ]:
# 2rv
mask1 = cek_gaia*cek_rave
mask2 = cek_gaia*cek_galah
mask3 = cek_gaia*cek_lamost
mask4 = cek_gaia*cek_apogee
mask5 = cek_galah*cek_rave
mask6 = cek_lamost*cek_galah
mask7 = cek_galah*cek_apogee
mask8 = cek_rave*cek_lamost
mask9 = cek_rave*cek_apogee
mask10 = cek_lamost*cek_apogee

mask = mask1 | mask2 | mask3 | mask4 | mask5 | mask6 | mask7 | mask8 | mask9 | mask10
mask.sum()

In [ ]:
rv2 = full[mask]
rv2.export("2rv.hdf5", progress=True)

In [ ]:
# 1rv
mask1 = cek_gaia
mask2 = cek_galah
mask3 = cek_lamost
mask4 = cek_apogee
mask5 = cek_rave

mask = mask1 | mask2 | mask3 | mask4 | mask5
mask.sum()

In [ ]:
rv1 = full[mask]
rv1.export("1rv.hdf5", progress=True)

### convert to fits?

In [ ]:
files = glob(<rv path>)
files.sort()

In [ ]:
# 1rv is big, very big.
# it's better to divide it into multiple RAs
ras = np.linspace(0,360, 19).astype(int)

In [ ]:
%%time
rv1 = vaex.open(<rv1 path>)
for i, (ra0, ra1) in enumerate(zip(ras[:-1], ras[1:])):
    if ra0 <= 80: continue
    rv_mask = rv1.filter(rv1.ra > ra0).filter(rv1.ra < ra1)
    print("filtered")
    rv_pandas = rv_mask.to_pandas_df()
    print(f"pandas-ed {ra0:03d}-{ra1:03d}")
    table = Table.from_pandas(rv_pandas)
    print("tabled ")
    table["subclass_lamost"] = table["subclass_lamost"].astype(str)
    # save into 1rv directory
    table.write(f"1rv/gaia-{ra0:03d}-{ra1:03d}.fits")

In [ ]:
# for other rvs, not necessary to divide them

In [ ]:
# files is rvs path ('1rv.hdf5', '2rv.hdf5', ...)
for file in files[1:]:
    rv = vaex.open(file)
    rv_pandas = rv.to_pandas_df()
    print(f"pandas-ed {file}")
    table = Table.from_pandas(rv_pandas)
    print("tabled ")
    table["subclass_lamost"] = table["subclass_lamost"].astype(str)
    # save it in fits format, strip '.hdf5' in file name
    table.write(f"{file[:3]}.fits")